## Build a Chatbot

https://js.langchain.com/docs/tutorials/chatbot

#### Quickstart

* Instantiate the model

In [7]:
// Load environment variables
import * as dotenv from "dotenv";
dotenv.config();
// Check if the OPENAI_API_KEY environment variable is set
if (!process.env.OPENAI_API_KEY) {
  throw new Error("Missing OPENAI_API_KEY environment variable");
}

import { ChatOpenAI } from "@langchain/openai";
const llm = new ChatOpenAI({
  model: "gpt-4o-mini",
  temperature: 0
});

In [8]:
await llm.invoke([{ role: "user", content: "Hi im bob" }]);

AIMessage {
  "id": "chatcmpl-BK0uDceHbtihYa1ZYasGxioqBGbXQ",
  "content": "Hi Bob! How can I assist you today?",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 10,
      "completionTokens": 11,
      "totalTokens": 21
    },
    "finish_reason": "stop",
    "model_name": "gpt-4o-mini-2024-07-18",
    "usage": {
      "prompt_tokens": 10,
      "completion_tokens": 11,
      "total_tokens": 21,
      "prompt_tokens_details": {
        "cached_tokens": 0,
        "audio_tokens": 0
      },
      "completion_tokens_details": {
        "reasoning_tokens": 0,
        "audio_tokens": 0,
        "accepted_prediction_tokens": 0,
        "rejected_prediction_tokens": 0
      }
    },
    "system_fingerprint": "fp_b376dfbbd5"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "output_tokens": 11,
    "input_tokens": 10,
    "total_tokens": 21,
    "input_token_details": {
      "audio": 0,
      "cache_read": 0
    },
  

The model on its own does not have any concept of state. For example, if you ask a followup question:

In [9]:
await llm.invoke([{ role: "user", content: "Whats my name" }]);

AIMessage {
  "id": "chatcmpl-BK0uDMip3uETmP4T6mpXLoq0wule7",
  "content": "I'm sorry, but I don't have access to personal information about you unless you've shared it in this conversation. How can I assist you today?",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 10,
      "completionTokens": 29,
      "totalTokens": 39
    },
    "finish_reason": "stop",
    "model_name": "gpt-4o-mini-2024-07-18",
    "usage": {
      "prompt_tokens": 10,
      "completion_tokens": 29,
      "total_tokens": 39,
      "prompt_tokens_details": {
        "cached_tokens": 0,
        "audio_tokens": 0
      },
      "completion_tokens_details": {
        "reasoning_tokens": 0,
        "audio_tokens": 0,
        "accepted_prediction_tokens": 0,
        "rejected_prediction_tokens": 0
      }
    },
    "system_fingerprint": "fp_b376dfbbd5"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "output_tokens": 29,
    "input_tokens":

To get around this, we need to pass the entire conversation history into the model. Let’s see what happens when we do that:

In [10]:
await llm.invoke([
  { role: "user", content: "Hi! I'm Bob" },
  { role: "assistant", content: "Hello Bob! How can I assist you today?" },
  { role: "user", content: "What's my name?" },
]);

AIMessage {
  "id": "chatcmpl-BK0uECRhnsB2jx0Ucd9ZQvbKU6QvD",
  "content": "Your name is Bob! How can I help you today?",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 33,
      "completionTokens": 13,
      "totalTokens": 46
    },
    "finish_reason": "stop",
    "model_name": "gpt-4o-mini-2024-07-18",
    "usage": {
      "prompt_tokens": 33,
      "completion_tokens": 13,
      "total_tokens": 46,
      "prompt_tokens_details": {
        "cached_tokens": 0,
        "audio_tokens": 0
      },
      "completion_tokens_details": {
        "reasoning_tokens": 0,
        "audio_tokens": 0,
        "accepted_prediction_tokens": 0,
        "rejected_prediction_tokens": 0
      }
    },
    "system_fingerprint": "fp_b376dfbbd5"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "output_tokens": 13,
    "input_tokens": 33,
    "total_tokens": 46,
    "input_token_details": {
      "audio": 0,
      "cache_read": 0
 

And now we can see that we get a good response!

This is the basic idea underpinning a chatbot’s ability to interact conversationally. So how do we best implement this?

#### Message persistence

In [11]:
import {
  START,
  END,
  MessagesAnnotation,
  StateGraph,
  MemorySaver,
} from "@langchain/langgraph";

// Define the function that calls the model
const callModel = async (state: typeof MessagesAnnotation.State) => {
  const response = await llm.invoke(state.messages);
  return { messages: response };
};

// Define a new graph
const workflow = new StateGraph(MessagesAnnotation)
  // Define the node and edge
  .addNode("model", callModel)
  .addEdge(START, "model")
  .addEdge("model", END);

// Add memory
const memory = new MemorySaver();
const app = workflow.compile({ checkpointer: memory });

In [12]:
import { v4 as uuidv4 } from "uuid";

const config = { configurable: { thread_id: uuidv4() } };

This enables us to support multiple conversation threads with a single application, a common requirement when your application has multiple users.

In [13]:
const input = [
  {
    role: "user",
    content: "Hi! I'm Bob.",
  },
];
const output = await app.invoke({ messages: input }, config);
// The output contains all messages in the state.
// This will log the last message in the conversation.
console.log(output.messages[output.messages.length - 1]);

AIMessage {
  "id": "chatcmpl-BK0uFjJyBtu8RIRoXYx0Mtlgk0vPU",
  "content": "Hi Bob! How can I assist you today?",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 12,
      "completionTokens": 11,
      "totalTokens": 23
    },
    "finish_reason": "stop",
    "model_name": "gpt-4o-mini-2024-07-18",
    "usage": {
      "prompt_tokens": 12,
      "completion_tokens": 11,
      "total_tokens": 23,
      "prompt_tokens_details": {
        "cached_tokens": 0,
        "audio_tokens": 0
      },
      "completion_tokens_details": {
        "reasoning_tokens": 0,
        "audio_tokens": 0,
        "accepted_prediction_tokens": 0,
        "rejected_prediction_tokens": 0
      }
    },
    "system_fingerprint": "fp_b376dfbbd5"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "output_tokens": 11,
    "input_tokens": 12,
    "total_tokens": 23,
    "input_token_details": {
      "audio": 0,
      "cache_read": 0
    },
  

In [15]:
const input2 = [
  {
    role: "user",
    content: "What's my name?",
  },
];
const output2 = await app.invoke({ messages: input2 }, config);
console.log(output2.messages[output2.messages.length - 1]);

AIMessage {
  "id": "chatcmpl-BK0zcsL9rkhawo6I1LOoSkoc1DYzl",
  "content": "Your name is Bob! How can I help you today, Bob?",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 34,
      "completionTokens": 15,
      "totalTokens": 49
    },
    "finish_reason": "stop",
    "model_name": "gpt-4o-mini-2024-07-18",
    "usage": {
      "prompt_tokens": 34,
      "completion_tokens": 15,
      "total_tokens": 49,
      "prompt_tokens_details": {
        "cached_tokens": 0,
        "audio_tokens": 0
      },
      "completion_tokens_details": {
        "reasoning_tokens": 0,
        "audio_tokens": 0,
        "accepted_prediction_tokens": 0,
        "rejected_prediction_tokens": 0
      }
    },
    "system_fingerprint": "fp_b376dfbbd5"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "output_tokens": 15,
    "input_tokens": 34,
    "total_tokens": 49,
    "input_token_details": {
      "audio": 0,
      "cache_read"

Great! Our chatbot now remembers things about us. If we change the config to reference a different thread_id, we can see that it starts the conversation fresh.

In [18]:
const config2 = { configurable: { thread_id: uuidv4() } };
const input3 = [
  {
    role: "user",
    content: "What's my name?",
  },
];
const output3 = await app.invoke({ messages: input3 }, config2);
console.log(output3.messages[output3.messages.length - 1]);

AIMessage {
  "id": "chatcmpl-BK11VBuuQ2E6dPaykDbs2sCuXsj2Q",
  "content": "I'm sorry, but I don't have access to personal information about you unless you've shared it in this conversation. How can I assist you today?",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 11,
      "completionTokens": 29,
      "totalTokens": 40
    },
    "finish_reason": "stop",
    "model_name": "gpt-4o-mini-2024-07-18",
    "usage": {
      "prompt_tokens": 11,
      "completion_tokens": 29,
      "total_tokens": 40,
      "prompt_tokens_details": {
        "cached_tokens": 0,
        "audio_tokens": 0
      },
      "completion_tokens_details": {
        "reasoning_tokens": 0,
        "audio_tokens": 0,
        "accepted_prediction_tokens": 0,
        "rejected_prediction_tokens": 0
      }
    },
    "system_fingerprint": "fp_b376dfbbd5"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "output_tokens": 29,
    "input_tokens":

However, we can always go back to the original conversation (since we are persisting it in a database)

In [20]:
const output4 = await app.invoke({ messages: input2 }, config);
console.log(output4.messages[output4.messages.length - 1]);

fetch failed

Context: trace=523b73ef-e356-4726-83a7-c622bf657ddc,id=523b73ef-e356-4726-83a7-c622bf657ddc; trace=523b73ef-e356-4726-83a7-c622bf657ddc,id=10ef5a1c-7b8e-487f-957c-6f7821628a1d; trace=523b73ef-e356-4726-83a7-c622bf657ddc,id=a7f05a3d-1203-4a75-99b2-21b481e5b1dc; trace=523b73ef-e356-4726-83a7-c622bf657ddc,id=2600a2a6-d006-413c-a824-6f286e6f888b; trace=523b73ef-e356-4726-83a7-c622bf657ddc,id=3e5261b8-b6b3-482d-af89-e4da579a0bd1; trace=523b73ef-e356-4726-83a7-c622bf657ddc,id=9a250aa1-1c5d-4075-bfc8-a2bc347ea730


AIMessage {
  "id": "chatcmpl-BK128fk79HCLwaLKDOQpPH7VBDXST",
  "content": "Your name is Bob. Is there anything else you'd like to talk about?",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 60,
      "completionTokens": 16,
      "totalTokens": 76
    },
    "finish_reason": "stop",
    "model_name": "gpt-4o-mini-2024-07-18",
    "usage": {
      "prompt_tokens": 60,
      "completion_tokens": 16,
      "total_tokens": 76,
      "prompt_tokens_details": {
        "cached_tokens": 0,
        "audio_tokens": 0
      },
      "completion_tokens_details": {
        "reasoning_tokens": 0,
        "audio_tokens": 0,
        "accepted_prediction_tokens": 0,
        "rejected_prediction_tokens": 0
      }
    },
    "system_fingerprint": "fp_b376dfbbd5"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "output_tokens": 16,
    "input_tokens": 60,
    "total_tokens": 76,
    "input_token_details": {
      "audio": 0,


This is how we can support a chatbot having conversations with many users!

Right now, all we’ve done is add a simple persistence layer around the model. We can start to make the more complicated and personalized by adding in a prompt template.

#### Prompt templates